# Loading the Linear regression with minibatch gradient descent model

Saving and restoring models tutorial: http://cv-tricks.com/tensorflow-tutorial/save-restore-tensorflow-models-quick-complete-tutorial/

In [1]:
import numpy as np
import tensorflow as tf
from sklearn.datasets import fetch_california_housing
from sklearn.preprocessing import StandardScaler

## Preparing data

loading data:

In [2]:
housing=fetch_california_housing()

scaling data (important in case of gradient descent):

In [3]:
m, n=housing.data.shape
stdsc=StandardScaler()
scaled_housing=stdsc.fit_transform(housing.data)
scaled_housing_plus_bias=np.c_[np.ones([m,1]),scaled_housing]

scaled_housing_plus_bias.mean(axis=0)

array([ 1.00000000e+00,  6.60969987e-17,  5.50808322e-18,  6.60969987e-17,
       -1.06030602e-16, -1.10161664e-17,  3.44255201e-18, -1.07958431e-15,
       -8.52651283e-15])

batchsize and number of batches to be processed in one epoch:

In [4]:
batchsize=100
n_batches=np.ceil(m/batchsize).astype(int)
n_batches

207

function for selection of a batch and parsing X, y:

In [5]:
def select_minibatch(epoch, batch_index, batch_size):
    np.random.seed(epoch*n_batches+batch_index)
    #print(epoch*n_batches+batch_index)
    indices=np.random.randint(m, size=batch_size)
    X=scaled_housing_plus_bias[indices]
    y=housing.target[indices]
    #print(indices)
    return X,y

X,y=select_minibatch(1, batchsize, 5)
X.shape

(5, 9)

## Restoring model and parameters

Importing computational graph (of one gradient descent step):

In [6]:
tf.reset_default_graph()
saver=tf.train.import_meta_graph("./saved_models/final_minibatch.ckpt.meta")

check that its the same graph:

Restoring all the variables (Tensors) nd operations (training_op):

In [7]:
sess=tf.Session()
saver.restore(sess, "./saved_models/final_minibatch.ckpt")
X = tf.get_default_graph().get_tensor_by_name("X:0")
y = tf.get_default_graph().get_tensor_by_name("y:0")
theta = tf.get_default_graph().get_tensor_by_name("Theta:0")
mse = tf.get_default_graph().get_tensor_by_name("mse:0")
training_op=tf.get_default_graph().get_operation_by_name("Training_op")

INFO:tensorflow:Restoring parameters from ./saved_models/final_minibatch.ckpt


Further training of the model. First element of theta in each epoch is printed. Note, that the first number is equal to the first element of final theta from the notebook where our model is constructed.

In [8]:
with sess:
    n_epochs=10
    for epoch in range(n_epochs):
        print(theta.eval()[0])
        for batch_index in range(n_batches):
            Xb,yb=select_minibatch(epoch, batch_index, batchsize)
            sess.run(training_op, feed_dict={X:Xb, y:yb.reshape(-1,1)})
    best_theta2=theta.eval(session=sess)
    print(theta.eval()[0])

[2.0654843]
[2.0696123]
[2.066603]
[2.069641]
[2.0701044]
[2.0707655]
[2.0730562]
[2.0609705]
[2.0746346]
[2.06418]
[2.0702066]


optimal theta for the scaled data:

In [9]:
best_theta2

array([[ 2.0702066 ],
       [ 0.83753353],
       [ 0.11625481],
       [-0.26712072],
       [ 0.34930074],
       [ 0.00257434],
       [-0.01061657],
       [-0.9086142 ],
       [-0.8744551 ]], dtype=float32)

check that the computational graph has, also after loading all the Tensor values still the same size as the graph in the notebook where our model is constructed (it should be so, as this should be the same graph).

In [10]:
gf=tf.get_default_graph()
len(gf.get_operations())

70